In [ ]:
import numpy as np
from scipy.signal import argrelextrema


import numpy as np
def moving_average(ts, win):
    return np.convolve(ts, np.ones(win) / win, mode="valid")


def peak_and_valley(bars):
    ma = moving_average(bars["close"], 10)
    
    peak_indexes = argrelextrema(ma, np.greater)
    peaks = peak_indexes[0]

    # Find valleys(min).
    valley_indexes = argrelextrema(ma, np.less)
    valleys = valley_indexes[0]

    assert abs(len(peaks) - len(valleys)) <= 1
    
    bars = bars[9:]
    # Plot main graph.
    (fig, ax) = plt.subplots()
    ax.plot(np.arange(len(bars)), bars["close"], color='c')
    ax.plot(np.arange(len(bars)), ma, color='b')

    # Plot peaks.
    peak_x = peaks
    peak_y = bars['close'][peak_x]
    ax.plot(peak_x, peak_y, 'gv', label="Peaks")

    # Plot valleys.
    valley_x = valleys
    valley_y = bars['close'][valley_x]
    ax.plot(valley_x, valley_y, 'r^', label="Valleys")
     
    trades = []
    gains = 1
    order = None

    vertex = sorted([*peaks, *valleys])
    for x in vertex:
        close = bars["close"]
        buy = x in valleys
        sell = x in peaks
        if buy and order is None:
            order = {
                "buy": close[x] * 0.5 + close[x+1] * 0.5,
                "buy_at": bars["frame"][x+1]
            }

        elif sell and order:
            buy = order["buy"]
            sell = close[x+1] * 0.5 + close[x] * 0.5,
            gain = sell / buy
            order.update({
                "sell": sell,
                "sell_at": bars["frame"][x+1],
                "gain": gain
            })

            gains *= gain
            trades.append(order)
            order = None

    return gains - 1, trades